In [6]:
!pip install pandas
!pip install csv


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\lisaa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement csv (from versions: none)
ERROR: No matching distribution found for csv

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\lisaa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [50]:
def weighting_educ(column_mf, column_level):
    #Creating the observed data
    possible_levels = {"HS-grad", "Some-college", "Assoc-acdm", "Assoc-voc", "Bachelors", "Masters", "Prof-school", "Doctorate"}
    p_obs_male_dict = {"8":0, "11":0, "HS-grad": 0, "Some-college": 0, "assoc": 0, "Bachelors": 0, "Masters": 0, "Prof-school": 0, "Doctorate": 0}
    p_obs_female_dict = {"8":0, "11":0, "HS-grad": 0, "Some-college": 0, "assoc": 0, "Bachelors": 0, "Masters": 0, "Prof-school": 0, "Doctorate": 0}
    female_ob = 0
    male_ob = 0
    for i,(level,gender) in enumerate(zip(column_level, column_mf)):
        #Counting all instances per category
        if not level in possible_levels:
            try:
                if level == "Preschool":
                    if gender == "Male":
                        p_obs_male_dict["8"] += 1
                    else:
                        p_obs_female_dict["8"] += 1
                elif int(level[0]) <= 8:
                    if gender == "Male":
                        p_obs_male_dict["8"] += 1
                    else:
                        p_obs_female_dict["8"] += 1
            except:
                print(level)
            else:
                if gender == "Male":
                    p_obs_male_dict["11"] += 1
                else:
                    p_obs_female_dict["11"] += 1  
        elif level in {"Assoc-acdm", "Assoc-voc"}:
            if gender == "Male":
                    p_obs_male_dict["assoc"] += 1
            else:
                p_obs_female_dict["assoc"] += 1
        else:
            if gender == "Male":
                p_obs_male_dict[level] += 1
            else:
                p_obs_female_dict[level] += 1
        #Counting the amount of men and women
        if gender == "Male":
            male_ob += 1
        else:
            female_ob += 1
    #Changing the dicts used to count into lists
    p_obs_male = [p_obs_male_dict["8"], p_obs_male_dict["11"],p_obs_male_dict["HS-grad"],p_obs_male_dict["Some-college"],p_obs_male_dict["assoc"],p_obs_male_dict["Bachelors"], p_obs_male_dict["Masters"], p_obs_male_dict["Doctorate"]] 
    p_obs_male = [x  / (male_ob + female_ob) * male_ob for x in p_obs_male]
    p_obs_female = [p_obs_female_dict["8"], p_obs_female_dict["11"],p_obs_female_dict["HS-grad"],p_obs_female_dict["Some-college"],p_obs_female_dict["assoc"],p_obs_female_dict["Bachelors"], p_obs_female_dict["Masters"], p_obs_female_dict["Doctorate"]]
    p_obs_female = [x  / (male_ob + female_ob) * female_ob for x in p_obs_female]
    
    #Calculating expected data using data from:
    #https://www.census.gov/data/tables/2022/demo/educational-attainment/cps-detailed-tables.html
    male = 109.979/226274
    female = 116.296/ 226274
    levels_male = [x /226274 for x in [4041,	6303,	33081,	16085,	10616,	25192,	10156,	1860,	2644] ]
    levels_female = [x /226274 for x in [3895,	5695,	31383,	16900,	13057,	27853,	13725,	1584,	2203] ]
    p_exp_male = []
    p_exp_female = []
    for i in range(len(levels_male)):
        p_exp_male.append(male * levels_male[i])
        p_exp_female.append(female * levels_female[i])
    
    #calculating the weights by deviding the expected by the observed. The output is in order of the dicts on the first lines of the function.
    weights_male = []
    weights_female = []
    for obs_m, exp_m, obs_f, exp_f in zip(p_obs_male, p_exp_male, p_obs_female, p_exp_female):
        weights_male.append(exp_m / obs_m)
        weights_female.append(exp_f / obs_f)
    
    return weights_male, weights_female
    

In [51]:
#Reweighing for marital status

In [52]:
# applying weights
import csv
gender = []
educ = []
with open('adult.csv', newline='') as data:
    spamreader = csv.reader(data, delimiter=' ')
    for row in spamreader:
        row = row[0].split(",")
        educ.append(row[3][1:-1])
        gender.append(row[9][1:-1])
male, female = weighting_educ(gender, educ)
print(male, female)

education
[5.06295450327839e-09, 6.900453614767138e-09, 1.493283444881931e-08, 1.1512056636894723e-08, 2.230138603324923e-08, 2.164462569072266e-08, 2.7464156991975184e-08, 1.8258264375444193e-08] [2.272171956131687e-08, 2.9600640399685614e-08, 6.356308316891667e-08, 4.135321068918174e-08, 9.734058906539846e-08, 1.1812323343637996e-07, 1.758157427013058e-07, 1.2646400231391827e-07]
